<a href="https://colab.research.google.com/github/BryanDYang/MOSA-Fall-Hackathon-2024/blob/main/Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>